In [1]:
//Project Document
//Muntaha Pasha, Xinyu Cao, Mattie Yang

//Imports
import $file.projectstdlib
import projectstdlib._
import scala.collection.mutable.ListBuffer

Compiling /Users/cassiecao/Desktop/CSCI 3155/Project/projectstdlib.sc

import $file.$            

import projectstdlib._

import scala.collection.mutable.ListBuffer

In [2]:
sealed trait Expr
case class Const(x : Integer) extends Expr
case class Bin(x : Bool) extends Expr
case class Ident(x : String) extends Expr
case class SymbolExpr(name : String) extends Expr
case class Plus(x : Expr, y : Expr) extends Expr
case class Minus(x : Expr, y : Expr) extends Expr
case class Mult(x : Expr, y : Expr) extends Expr
case class Pow(x : Expr, y : Expr) extends Expr
case class Neg(x : Expr) extends Expr
case class Eq(x : Expr, y : Expr) extends Expr
case class And(x : Expr, y : Expr) extends Expr
case class Or(x : Expr, y : Expr) extends Expr
case class IfThenElse(p : Expr, t : Expr, f : Expr) extends Expr
case class Define(id : String, args: List[Expr], body : Expr) extends Expr  
case class Pair(x : Expr, xs : Expr) extends Expr

defined trait Expr
defined class Const
defined class Bin
defined class Ident
defined class SymbolExpr
defined class Plus
defined class Minus
defined class Mult
defined class Pow
defined class Neg
defined class Eq
defined class And
defined class Or
defined class IfThenElse
defined class Define
defined class Pair

In [3]:
sealed trait Environment
case object EmptyEnv extends Environment 
case class Extend(k : String, v : Value, env : Environment) extends Environment

sealed trait Value
case object Error extends Value
case object Plus_ extends Value
case object Minus_ extends Value
case object Mult_ extends Value
case object Pow_ extends Value
case object Neg_ extends Value
case object Eq_ extends Value
case object And_ extends Value
case object Or_ extends Value
case object If_ extends Value
case class NumVal(x : Integer) extends Value
case class BinVal(x : Bool) extends Value
case class ClosureSimple(args: List[Expr], body: Expr, env: Environment) extends Value

defined trait Environment
defined object EmptyEnv
defined class Extend
defined trait Value
defined object Error
defined object Plus_
defined object Minus_
defined object Mult_
defined object Pow_
defined object Neg_
defined object Eq_
defined object And_
defined object Or_
defined object If_
defined class NumVal
defined class BinVal
defined class ClosureSimple

In [4]:
def lookupEnv(sigma: Environment, x: String): Maybe[Value] = sigma match {
    case EmptyEnv => Nothing
    case Extend(y, v, pi) => string_eq(y,x) match {
        case True => Just(v)
        case False => lookupEnv(pi, x)
    } 
}   

defined function lookupEnv

In [5]:
def eval(env : Environment, expr : Expr) : Value = expr match {
    case Const(n)  => NumVal(n)
    case Bin(p)    => BinVal(p)
    case Ident(id) => lookupEnv(env, id) match {
        case Just(v) => v
        case Nothing => Error
    }
    case SymbolExpr(x) => x match {
        case "+" => Plus_
        case "-" => Minus_
        case "*" => Mult_
        case "expt" => Pow_
        case "not" => Neg_
        case "eq" => Eq_
        case "and" => And_
        case "or" => Or_
        case "if" => If_
    }
    case Pow(e1, e2)  => (eval(env, e1), eval(env,e2)) match {
        case (NumVal(x), NumVal(Positive(n))) => NumVal(pow(x, n))
        case _ => Error
    }
    case Plus(e1, e2) => eval_bin_arith(plus, env, e1, e2)
    case Minus(e1, e2) => eval_bin_arith(minus, env, e1, e2)
    case Mult(e1, e2) => eval_bin_arith(mult, env, e1, e2)
    case Neg(e) => eval(env, e) match {
        case NumVal(x) => NumVal(negate(x))
        case BinVal(p) => BinVal(not(p))
        case _     => Error
    }
    case Eq(e1, e2) => (eval(env, e1), eval(env, e2)) match {
        case (NumVal(x), NumVal(y)) => BinVal(int_eq(x,y))
        case (BinVal(p), BinVal(q)) => BinVal(bool_eq(p,q))
        case _                      => Error
    }
    case And(e1, e2) => eval_bin_bool(and, env, e1, e2)
    case Or(e1, e2) => eval_bin_bool(or, env, e1, e2)
    case IfThenElse(p, e_t, e_f) => eval(env, p) match{
        case BinVal(True)  => eval(env, e_t)
        case BinVal(False) => eval(env, e_f)
        case _             => Error
    }
    case Define(id, args, body) => args match {
        case Empty => {
            val new_env = Extend(id,eval(env,body),env)
            eval(new_env,body)
        }
        case _ => {
            val new_env = Extend(id,ClosureSimple(args,body,env),env)
            ClosureSimple(args,body,env)
        }
    }
    case Pair(x,y) => (eval(env,x),y) match {
        case (Plus_,Pair(e1,e2)) => eval(env,Plus(e1,e2))
        case (Minus_,Pair(e1,e2)) => eval(env,Minus(e1,e2))
        case (Mult_,Pair(e1,e2)) => eval(env,Mult(e1,e2))
        case (Pow_,Pair(e1,e2)) => eval(env,Pow(e1,e2))
        case (Neg_,e1) => eval(env,Neg(e1))
        case (Eq_,Pair(e1,e2)) => eval(env,Eq(e1,e2))
        case (And_,Pair(e1,e2)) => eval(env,And(e1,e2))
        case (Or_, Pair(e1,e2)) => eval(env,Or(e1,e2))
        case (If_, Pair(e1,Pair(e2,e3))) => eval(env,IfThenElse(e1,e2,e3))
    }
}

def eval_bin_arith( op : (Integer, Integer) => Integer
                  , env : Environment
                  , e1 : Expr
                  , e2 : Expr) : Value 
    = (eval(env, e1), eval(env, e2)) match{
        case (NumVal(x), NumVal(y)) => NumVal(op(x,y))
        case _ => Error
    }

def eval_bin_bool( op : (Bool, Bool) => Bool
                 , env : Environment
                 , e1 : Expr
                 , e2 : Expr) : Value 
    = (eval(env, e1), eval(env, e2)) match{
        case (BinVal(x), BinVal(y)) => BinVal(op(x,y))
        case _ => Error
    }

defined function eval
defined function eval_bin_arith
defined function eval_bin_bool

In [6]:
def parse(source : String) = {
    val parsed = source.split("(?<![a-z,A-Z,-?\\d])|(?![a-z,A-Z,-?\\d])").filter(_!=" ").toList
    println(parsed)
    var num_open = 0
    var num_paren = 0
    
    var args = new ListBuffer[Expr]()
    val exprs = new ListBuffer[Expr]()
    val sym = List("+","-","*","expt","not","eq","and","or","if")
    val letters = List("a","b","c","d","e","f","g","h","i","j","k","l","m","n","o","p","q","r","s","t","u","v","w","x","y","z")
    
    if (parsed.contains("define")){
        for (i <- (parsed.indexOf("define")+1) to (parsed.length - 1)) {
            if (parsed(i) == "(") {
                if (num_open == 0){
                    num_open += 1
                }
            }
            else if (parsed(i) == ")") {
                if (num_open != 0){
                    num_open -= 1

                    if (num_open == 0){
                        num_paren += 1
                    }
                }
            }
        }

        
        // form args and body
        val id : String = parsed(parsed.indexOf("define")+1)
            
        if (num_paren == 1) {
            //args will be empty like whe n it is defined
            
            //body will contain Pair(+, Pair(5, Pair(-, Pair(2 3))))
            //change strings into Expr, put into List exprs
            for (j <- (parsed.length-1) to (parsed.indexOf("define")+2) by -1){
                if (parsed(j) != "(" & parsed(j) != ")"){
                    if (sym.contains(parsed(j))){
                        val symbol_elem : Expr = SymbolExpr(parsed(j))
                        exprs += symbol_elem
                    }
                    else if(parsed(j) == "True" || parsed(j) == "False"){
                        val tf_elem : Expr = Bin(string_to_bool(parsed(j)))
                        exprs += tf_elem
                    }
                    else{
                        val num_elem : Expr = Const(string_to_int(parsed(j)))
                        exprs += num_elem
                    }
                }
            }
        }
        else if (num_paren == 2) {
            var paren_close = 0
            var ind = parsed.indexOf("define") + 2
            
            //args will be List(SymbolExpr(x), SymbleExpr(y))
            //change strings into Expr, put into List args
            while (paren_close == 0){
                if (parsed(ind) == "("){
                    ind += 1
                }
                else if (parsed(ind) == ")"){
                    paren_close += 1
                }
                else {
                    val var_elem : Expr = SymbolExpr(parsed(ind))
                    args += var_elem
                    ind += 1
                }
            }
            
            
            //body will be Pair(+, Pair(x y))
            //change strings into Expr, put into List exprs
            for (j <- parsed.length-1 to ind+1 by -1){
                if (parsed(j) != "(" & parsed(j) != ")"){
                    if (letters.contains(parsed(j)) || sym.contains(parsed(j))){
                        val sym_elem : Expr = SymbolExpr(parsed(j))
                        exprs += sym_elem
                    }
                    else if(parsed(j) == "True" || parsed(j) == "False"){
                        val tf_elem : Expr = Bin(string_to_bool(parsed(j)))
                        exprs += tf_elem
                    }
                    else{
                        val num_elem : Expr = Const(string_to_int(parsed(j))) 
                        exprs += num_elem
                    }
                }
            }
        }
        
        // change exprs to body
        // Form Pair(..,Pair(..,Pair(...,...)))
        var pair : Expr = null
        var body : Expr = null
        if (exprs.contains(SymbolExpr("not"))){
            body = Pair(exprs(1),exprs(0))
        }
        else{
            pair = Pair(exprs(1),exprs(0))
            body = Pair(exprs(2),pair)
            for (i <- 3 to exprs.length-1){
                body = Pair(exprs(i),body)
                pair = body
            }
        }
        
        // call eval with Define(id,args,body)
        eval(EmptyEnv, Define(id,scList_to_List(args.toList),body))
    }
    else {
        //no id
        //empty args
        
        //change strings into Expr, put into List exprs
        for (i <- parsed.length-1 to 0 by -1){
            if (parsed(i) != "(" & parsed(i) != ")"){
                if (sym.contains(parsed(i))){
                    val sym_elem : Expr = SymbolExpr(parsed(i))
                    exprs += sym_elem
                }
                else if(parsed(i) == "True" || parsed(i) == "False"){
                    val tf_elem : Expr = Bin(string_to_bool(parsed(i)))
                    exprs += tf_elem
                }
                else{
                    val num_elem : Expr = Const(string_to_int(parsed(i)))
                    exprs += num_elem
                }
            }
        }
        
        //change exprs to body
        //Form Pair(..,Pair(..,Pair(...,...)))
        var pair : Expr = null
        var body : Expr = null
        if (exprs.contains(SymbolExpr("not"))){
            body = Pair(exprs(1),exprs(0))
        }
        else{
            pair = Pair(exprs(1),exprs(0))
            body = Pair(exprs(2),pair)
            for (i <- 3 to exprs.length-1){
                body = Pair(exprs(i),body)
                pair = body
            }
        }
        
        //don't call eval with Define, instead just call eval with Pair(+, Pair(5, Pair(-, Pair(2 3))))
        eval(EmptyEnv,body)
    }
}

defined function parse

In [7]:
println("Test Case 1")
parse("(define f (* 5 (- 3 (expt (1 5)))))")
println("Test Case 2")
parse("(define f (- 6 (expt -2 2)))")
println("Test Case 3")
parse("(define f (x y z) (- 5 (+ z(* (x y))))")
println("Test Case 4")
parse("(define f (x y) (+ 2 -3)")
println("Test Case 5")
parse("(+ 5 ( - (2 3)))")
println("Test Case 6")
parse("(- -5 (* -2 3))")
println("Test Case 7")
parse("(eq (1 5))")
println("Test Case 8")
parse("(eq (True True))")
println("Test Case 9")
parse("(define f (eq (-1 -1)))")
println("Test Case 10")
parse("(not 5)")
println("Test Case 11")
parse("(not -3)")
println("Test Case 12")
parse("(define f (not True))")
println("Test Case 13")
parse("(define func (not -6))")
println("Test Case 14")
parse("(and (True False)")
println("Test Case 15")
parse("(define f (and (True (eq (3 3)))))")
println("Test Case 16")
parse("(or (False False))")
println("Test Case 17")
parse("(define Y (or (False (eq (3 3)))))")
println("Test Case 18")
parse("(if (True (5 2)))")
println("Test Case 19")
parse("(if (True (-1 -2)))")
println("Test Case 20")
parse("(define func (if (False (1 0))))")

Test Case 1
List((, define, f, (, *, 5, (, -, 3, (, expt, (, 1, 5, ), ), ), ), ))
Test Case 2
List((, define, f, (, -, 6, (, expt, -2, 2, ), ), ))
Test Case 3
List((, define, f, (, x, y, z, ), (, -, 5, (, +, z, (, *, (, x, y, ), ), ), ))
Test Case 4
List((, define, f, (, x, y, ), (, +, 2, -3, ))
Test Case 5
List((, +, 5, (, -, (, 2, 3, ), ), ))
Test Case 6
List((, -, -5, (, *, -2, 3, ), ))
Test Case 7
List((, eq, (, 1, 5, ), ))
Test Case 8
List((, eq, (, True, True, ), ))
Test Case 9
List((, define, f, (, eq, (, -1, -1, ), ), ))
Test Case 10
List((, not, 5, ))
Test Case 11
List((, not, -3, ))
Test Case 12
List((, define, f, (, not, True, ), ))
Test Case 13
List((, define, func, (, not, -6, ), ))
Test Case 14
List((, and, (, True, False, ))
Test Case 15
List((, define, f, (, and, (, True, (, eq, (, 3, 3, ), ), ), ), ))
Test Case 16
List((, or, (, False, False, ), ))
Test Case 17
List((, define, Y, (, or, (, False, (, eq, (, 3, 3, ), ), ), ), ))
Test Case 18
List((, if, (, True, (, 5, 2,

res6_1: Value = NumVal(
  Positive(Succ(Succ(Succ(Succ(Succ(Succ(Succ(Succ(Succ(Succ(Zero)))))))))))
)
res6_3: Value = NumVal(Positive(Succ(Succ(Zero))))
res6_5: Value = ClosureSimple(
  Cons(SymbolExpr("x"), Cons(SymbolExpr("y"), Cons(SymbolExpr("z"), Empty))),
  Pair(
    SymbolExpr("-"),
    Pair(
      Const(Positive(Succ(Succ(Succ(Succ(Succ(Zero))))))),
      Pair(
        SymbolExpr("+"),
        Pair(
          SymbolExpr("z"),
          Pair(SymbolExpr("*"), Pair(SymbolExpr("x"), SymbolExpr("y")))
        )
      )
    )
  ),
  EmptyEnv
)
res6_7: Value = ClosureSimple(
  Cons(SymbolExpr("x"), Cons(SymbolExpr("y"), Empty)),
  Pair(
    SymbolExpr("+"),
    Pair(
      Const(Positive(Succ(Succ(Zero)))),
      Const(Negative(Succ(Succ(Succ(Zero)))))
    )
  ),
  EmptyEnv
)
res6_9: Value = NumVal(Positive(Succ(Succ(Succ(Succ(Zero))))))
res6_11: Value = NumVal(Positive(Succ(Zero)))
res6_13: Value = BinVal(False)
res6_15: Value = BinVal(True)
res6_17: Value = BinVal(True)
res6_19: Va